1994. 好子集的数目
给你一个整数数组 nums 。如果 nums 的一个子集中，所有元素的乘积可以表示为一个或多个 互不相同的质数 的乘积，那么我们称它为 好子集 。
比方说，如果 nums = [1, 2, 3, 4] ：
[2, 3] ，[1, 2, 3] 和 [1, 3] 是 好 子集，乘积分别为 6 = 2*3 ，6 = 2*3 和 3 = 3 。
[1, 4] 和 [4] 不是 好 子集，因为乘积分别为 4 = 2*2 和 4 = 2*2 。
请你返回 nums 中不同的 好 子集的数目对 109 + 7 取余 的结果。
nums 中的 子集 是通过删除 nums 中一些（可能一个都不删除，也可能全部都删除）元素后剩余元素组成的数组。
如果两个子集删除的下标不同，那么它们被视为不同的子集。

## 做法
可以将 $[1, 30]$ 中的整数分成三类：
* 对于任意一个好子集而言，添加任意数目的 1，得到的新子集仍是好子集
* 2,3,5,6,7,10,11,13,14,15,17,19,21,22,23,26,29,30 这些数不包含平方因子，因此每个数在好子集中至多出现一次
* 4,8,9,12,16,18,20,24,25,27,284,8,9,12,16,18,20,24,25,27,28 这些数包含平方因子，不能在好子集中出现。

可以通过硬编码把 $[1, 30]$ 中的整数按照上述分类，也可以先预处理出所有 $[1, 30]$ 中的质数 2,3,5,7,11,13,17,19,23,29，在通过试除的方式动态分类。
分类完成后，考虑动态规划。由于每个质因数只能出现一次，并且 $[1, 30]$ 中一共有 10 个质数，因此我们可以用一个长度为 10 的二进制数 mask 表示质因数的使用情况，其中第 i 位为 1 当且仅当第 i 个质数已经被使用过。
这样，定义 $f[i][mask]$ 表示当只选择 $[2,i]$ 范围的数，并且选择的数的质因数使用情况为 mask 时的方案数。如果 i 本身包含平方因子，那么无法选择 i ，相当于在 $[2,i-1]$ 范围内选择，状态转移方程为：$f[i][mask] = f[i - 1][mask]$。
如果 i 本身不包含平方因子，记其包含的质因子的二进制表示为 subset，那么状态转移方程为：$f[i][mask]=f[i-1][mask]+f[i-1][mask \backslash subset] \times freq[i]$。
其中：
* freq[i] 表示数组中 i 的出现次数
* $mask \backslash subset$ 表示从二进制表示 mask 中去除所有在 subset 中出现的 1，可以使用按位异或运算实现。需要保证 subset 是 mask 的子集，可以使用按位与运算判断。

边界条件为：$f[1][0] = 2^{freq[1]}$。即每一个在数组 nums 中出现的 1 都可以选或不选。最终的答案为所有 $f[30][..]$ 中去除 $f[30][0]$ 以外的项的总和。

In [ ]:
import collections
from typing import *

def numberOfGoodSubsets(nums: List[int]) -> int:
    primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
    mod = 10**9 + 7
    freq = collections.Counter(nums)
    f = [0] * (1 << len(primes))
    f[0] = pow(2, freq[1], mod)
    for i, occ in freq.items():
        if i == 1:
            continue
        # 检查 i 的每个质因数是否均不超过 1 个
        subset, x = 0, i
        check = True
        for j, prime in enumerate(primes):
            if x % (prime * prime) == 0:
                check = False
                break
            if x % prime == 0:
                subset |= (1 << j)
        if not check:
            continue
        # 动态规划
        for mask in range((1 << len(primes)) - 1, 0, -1):
            if (mask & subset) == subset:
                f[mask] = (f[mask] + f[mask ^ subset] * occ) % mod
    result = sum(f[1:]) % mod
    return result